In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "e1def575",
   "metadata": {},
   "source": [
    "# Data Challenge - Advanced Machine Learning\n",
    "\n",
    "\n",
    "**The data challenge is proposed by Idemia**\n",
    "\n",
    "## Description of the data challenge\n",
    "\n",
    "You have at your disposal 100000 images of human faces, and their occlusion label.\n",
    "The goal of this challenge is to regress the percentage of the face that is occluded.\n",
    "We also want to have similar performances on female and male, the gender label is given for the train database.\n",
    "\n",
    "Below is the formula of the evaluation score. You'll first need to compute the error made by your model for men and women separetely. The error is measured as follows\n",
    "\n",
    "$$\n",
    " Err = \\frac{\\sum_{i}{w_i(p_i - GT_i)^2}}{\\sum_{i}{w_i}}, w_i = \\frac{1}{30} + GT_i,\n",
    "$$\n",
    "where $GT_i$ is the ground truth and p_i the prediction made by your model. \n",
    "\n",
    "Then, the final score for the leaderboard is given by\n",
    "\n",
    "$$\n",
    "Score = \\frac{Err_F + Err_M}{2} + \\left | Err_F - Err_M \\right |,\n",
    "$$\n",
    "where $Err_F$ and $Err_M$ are the errors obtained for female and men, respectively.\n",
    "\n",
    "Hereafter we show you how to load the data and run a naive baseline using a pretrained model."
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "6d771bd4",
   "metadata": {},
   "source": [
    "### Downloading the Data:\n",
    "\n",
    "You can download the Dataset from the below links: https://partage.imt.fr/index.php/s/mscQABX4oZxx7ax"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "14a84fac-1388-46c0-8b2b-c5da6383b941",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries to run the naive baseline\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from PIL import Image\n",
    "from tqdm import tqdm\n",
    "from collections import OrderedDict\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "import torchvision\n",
    "import torchvision.transforms as transforms\n",
    "from torchvision.models import mobilenet_v3_small"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cfb931c0-cd9d-4b23-8d96-1cfb9706c3de",
   "metadata": {},
   "source": [
    "### Load dataframes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8e8dbe42-0b3e-4649-8aa8-2c2ce3b8d7d9",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_train = pd.read_csv(\"listes_training/data_100K/train_100K.csv\", delimiter=' ')\n",
    "df_test = pd.read_csv(\"listes_training/data_100K/test_students.csv\", delimiter=' ')\n",
    "\n",
    "image_dir = \"crops_100K\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "98b7e12a-f0f1-4779-9205-f4b7d44b84a1",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_train.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "18b973ad-10c6-44fd-bc23-3fbb644f12d7",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_test.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e2225ac6-a74d-43f7-a5a2-dae2a8bd5728",
   "metadata": {},
   "source": [
    "#### Remove nan values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4aaef2a2-4512-460b-af97-f91cece13caf",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_train = df_train.dropna()\n",
    "df_test = df_test.dropna()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7b6835d8-2964-4822-bd1c-e15fba297e42",
   "metadata": {},
   "source": [
    "### Split Dataframe in train and val"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "34261442-22d9-4611-9a65-1ccc5fbc4bff",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_val = df_train.loc[:20000].reset_index()\n",
    "df_train = df_train.loc[20000:].reset_index()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "309660aa-d8a9-404a-b7d7-5f4bb5dac72a",
   "metadata": {},
   "outputs": [],
   "source": [
    "len(df_train), len(df_val), len(df_test)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "925365d9-57c3-40e6-afe6-f555c9ebaa7c",
   "metadata": {},
   "source": [
    "### Check that all images are read correctly"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a13e82aa-db54-4961-b391-ea7d4721a377",
   "metadata": {},
   "outputs": [],
   "source": [
    "for idx, row in tqdm(df_train.iterrows(), total=len(df_train)):\n",
    "    try:\n",
    "        filename = df_train.loc[idx, 'filename']\n",
    "        img2display = Image.open(f\"{image_dir}/{filename}\")\n",
    "    except ValueError as e:\n",
    "        print(idx, e)\n",
    "        \n",
    "for idx, row in tqdm(df_test.iterrows(), total=len(df_test)):\n",
    "    try:\n",
    "        filename = df_test.loc[idx, 'filename']\n",
    "        img2display = Image.open(f\"{image_dir}/{filename}\")\n",
    "    except ValueError as e:\n",
    "        print(idx, e)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1f0f4aaa-05d3-4c47-a01e-415781e4ba72",
   "metadata": {},
   "source": [
    "### Display first images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a6518d04-f6e5-4d5c-a51e-b34711825f54",
   "metadata": {},
   "outputs": [],
   "source": [
    "for idx, row in df_train[:5].iterrows():\n",
    "    filename = row['filename']\n",
    "    occlusion = row['FaceOcclusion']\n",
    "    gender = row['gender']\n",
    "    img2display = Image.open(f\"{image_dir}/{filename}\")\n",
    "    display(img2display)\n",
    "    print(filename, occlusion, gender)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1465fe98-f578-4d97-bae1-b1fcaa59183c",
   "metadata": {},
   "source": [
    "### Make Dataset and Dataloader"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b580142f-57ae-4a85-9185-340871f46581",
   "metadata": {},
   "outputs": [],
   "source": [
    "class Dataset(torch.utils.data.Dataset):\n",
    "    'Characterizes a dataset for PyTorch'\n",
    "    def __init__(self, df, image_dir):\n",
    "         'Initialization'\n",
    "         self.image_dir = image_dir\n",
    "         self.df = df\n",
    "         self.transform = transforms.ToTensor()\n",
    "         \n",
    "    def __len__(self):\n",
    "        'Denotes the total number of samples'\n",
    "        return len(self.df)\n",
    "\n",
    "    def __getitem__(self, index):\n",
    "        'Generates one sample of data'\n",
    "        # Select sample\n",
    "        row = self.df.loc[index]\n",
    "        filename = row['filename']\n",
    "\n",
    "        # Load data and get label\n",
    "        img = Image.open(f\"{image_dir}/{filename}\")\n",
    "        y = row['FaceOcclusion']\n",
    "        gender = row['gender']\n",
    "\n",
    "        X = self.transform(img)\n",
    "        y = np.float32(y)\n",
    "\n",
    "        return X, y, gender, filename"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "00f9f263-4ae8-4922-b34a-4ce26e80c670",
   "metadata": {},
   "outputs": [],
   "source": [
    "training_set = Dataset(df_train, image_dir)\n",
    "validation_set = Dataset(df_val, image_dir)\n",
    "\n",
    "params_train = {'batch_size': 8,\n",
    "          'shuffle': True,\n",
    "          'num_workers': 0}\n",
    "\n",
    "params_val = {'batch_size': 8,\n",
    "          'shuffle': False,\n",
    "          'num_workers': 0}\n",
    "\n",
    "training_generator = torch.utils.data.DataLoader(training_set, **params_train)\n",
    "validation_generator = torch.utils.data.DataLoader(validation_set, **params_val)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "660e6c36-32e9-4f49-a655-7a4ff474f9df",
   "metadata": {},
   "source": [
    "### Create naive model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5d1400b5-5c75-4512-9b55-3a49b7de99e2",
   "metadata": {},
   "outputs": [],
   "source": [
    "model = torchvision.models.mobilenet_v3_small(num_classes=1)\n",
    "if torch.cuda.is_available():\n",
    "    model.cuda()    \n",
    "model"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "11c518a3-da5d-4ab4-aafe-17e4a4bbbf0a",
   "metadata": {},
   "source": [
    "### Loss and optimizer"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8e6c197d-1b93-405f-a2b3-ead6027e33f1",
   "metadata": {},
   "outputs": [],
   "source": [
    "loss_fn = nn.MSELoss()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ef7c0bc4-5415-4121-b22d-cba57ff7fe5f",
   "metadata": {},
   "outputs": [],
   "source": [
    "optimizer = torch.optim.Adam(model.parameters(), lr=0.001)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c317a740-179b-4056-a9c9-fee96bd151c7",
   "metadata": {},
   "source": [
    "### Train naive model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "cbf12e7c-c57f-4cac-bcc2-d244404a7d67",
   "metadata": {},
   "outputs": [],
   "source": [
    "# CUDA for PyTorch\n",
    "use_cuda = torch.cuda.is_available()\n",
    "device = torch.device(\"cuda:0\" if use_cuda else \"cpu\")\n",
    "torch.backends.cudnn.benchmark = True"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0aa6829d-8085-4e1a-b3d5-b24be1be83d4",
   "metadata": {},
   "source": [
    "#### Fit on train split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f38bfdf0-4dad-42c2-9d28-3cea32e07f94",
   "metadata": {},
   "outputs": [],
   "source": [
    "num_epochs = 1\n",
    "\n",
    "for n in range(num_epochs):\n",
    "    print(f\"Epoch {n}\")\n",
    "    for batch_idx, (X, y, gender, filename) in tqdm(enumerate(training_generator), total=len(training_generator)):\n",
    "        # Transfer to GPU\n",
    "        X, y = X.to(device), y.to(device)\n",
    "        y = torch.reshape(y, (len(y), 1))\n",
    "        y_pred = model(X)\n",
    "        loss = loss_fn(y_pred, y)\n",
    "\n",
    "        if loss.isnan():\n",
    "            print(filename)\n",
    "            print('label', y)\n",
    "            print('y_pred', y_pred)\n",
    "            break\n",
    "\n",
    "        if batch_idx % 200 == 0:\n",
    "            print(loss)\n",
    "        \n",
    "        optimizer.zero_grad()\n",
    "        loss.backward()\n",
    "        optimizer.step()\n",
    "        "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "23d57a8f-eb3b-4e51-9742-e551632a5e1b",
   "metadata": {},
   "source": [
    "## Evaluate metric on validation split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "63cb13ef-c632-4218-81d1-4044398bcd34",
   "metadata": {},
   "outputs": [],
   "source": [
    "def error_fn(df):\n",
    "    pred = df.loc[:, \"pred\"]\n",
    "    ground_truth = df.loc[:, \"target\"]\n",
    "    weight = 1/30 + ground_truth\n",
    "\n",
    "    return np.sum(((pred - ground_truth)**2) * weight, axis=0) / np.sum(weight, axis=0)\n",
    "\n",
    "def metric_fn(female, male):\n",
    "    err_male = error_fn(male)\n",
    "    err_female = error_fn(female)\n",
    "    return (err_male + err_female) / 2 + abs(err_male - err_female)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "63d530c4-59ce-4a8b-9936-260c46ef5cea",
   "metadata": {},
   "outputs": [],
   "source": [
    "results_list = []\n",
    "for batch_idx, (X, y, gender, filename) in tqdm(enumerate(validation_generator), total=len(validation_generator)):\n",
    "        X, y = X.to(device), y.to(device)\n",
    "        y_pred = model(X)\n",
    "        for i in range(len(X)):\n",
    "\n",
    "            results_list.append({'pred': float(y_pred[i]),\n",
    "                                  'target': float(y[i]),\n",
    "                                  'gender': float(gender[i])\n",
    "                                 })\n",
    "results_df = pd.DataFrame(results_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "53a2ee7d-f0d6-46c3-96e8-8a6f11208b4e",
   "metadata": {},
   "outputs": [],
   "source": [
    "results_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4dd58f48-bd46-4d2d-a291-9b6af4e5e57e",
   "metadata": {},
   "outputs": [],
   "source": [
    "results_df['pred'].hist()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "063209ae-d673-4096-a8eb-487dd14b2ba9",
   "metadata": {},
   "outputs": [],
   "source": [
    "results_male = results_df.loc[results_df[\"gender\"] > 0.5]\n",
    "results_female = results_df.loc[results_df[\"gender\"] < 0.5]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1e8ebfae-8101-4b2c-bec5-9b60845f453f",
   "metadata": {},
   "outputs": [],
   "source": [
    "metric_fn(results_male, results_female)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4006c9fd",
   "metadata": {},
   "source": [
    "## Evaluating your Test DataSet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d5edd7ce-e687-433c-8248-0dd4d7a6f21a",
   "metadata": {},
   "outputs": [],
   "source": [
    "class Dataset_test(torch.utils.data.Dataset):\n",
    "    'Characterizes a dataset for PyTorch'\n",
    "    def __init__(self, df, image_dir):\n",
    "         'Initialization'\n",
    "         self.image_dir = image_dir\n",
    "         self.df = df\n",
    "         self.transform = transforms.ToTensor()\n",
    "         \n",
    "    def __len__(self):\n",
    "        'Denotes the total number of samples'\n",
    "        return len(self.df)\n",
    "\n",
    "    def __getitem__(self, index):\n",
    "        'Generates one sample of data'\n",
    "        # Select sample\n",
    "        row = self.df.loc[index]\n",
    "        filename = row['filename']\n",
    "\n",
    "        # Load data and get label\n",
    "        img = Image.open(f\"{image_dir}/{filename}\")\n",
    "\n",
    "        X = self.transform(img)\n",
    "\n",
    "        return X, filename"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "60cfdfef",
   "metadata": {},
   "outputs": [],
   "source": [
    "test_set = Dataset_test(df_test, image_dir)\n",
    "\n",
    "params_test = {'batch_size': 8,\n",
    "          'shuffle': False,\n",
    "          'num_workers': 0}\n",
    "\n",
    "test_generator = torch.utils.data.DataLoader(test_set, **params_test)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9af5bcf6",
   "metadata": {},
   "outputs": [],
   "source": [
    "results_list = []\n",
    "for batch_idx, (X, filename) in tqdm(enumerate(test_generator), total=len(test_generator)):\n",
    "        X = X.to(device)\n",
    "        y_pred = model(X)\n",
    "        for i in range(len(X)):\n",
    "\n",
    "            results_list.append({'pred': float(y_pred[i])\n",
    "                                 })\n",
    "test_df = pd.DataFrame(results_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a8dbca68",
   "metadata": {},
   "outputs": [],
   "source": [
    "test_df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0a67de71",
   "metadata": {},
   "source": [
    "### Preparing your file for Submission"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a6d8c3b3",
   "metadata": {},
   "outputs": [],
   "source": [
    "test_df.to_csv(\"Data_Challenge.csv\", header = None, index = None)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "88ff8155",
   "metadata": {},
   "source": [
    "#### Now it's your turn. Good luck !  :) "
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}